# Logistic Regression & Gradient Descent
This demo implments a Logistic Regression (LR) model and illustrates how to solve LR via Gradient Descent (GD).

In [72]:
%matplotlib inline

import math
import numpy as np
import matplotlib.pyplot as plt
from pdb import set_trace as st
from sklearn.datasets import load_breast_cancer
from sklearn.preprocessing import StandardScaler

# Load the breast cancer wisconsin dataset for binary classification
X, y = load_breast_cancer(return_X_y=True)
N,D = X.shape
assert len(X)==len(y), print('inequal # of samples and labels')
print('This data set is given with {} sampels and {} features'.format(N, D))
print('Total class # is {}'.format(len(set(y))))
print(set(y))
# Feature normalization is helpful for classification
print(X[:3,:5])
scaler = StandardScaler().fit(X)
X_n = scaler.transform(X)
print(X_n[:3,:5])

This data set is given with 569 sampels and 30 features
Total class # is 2
{0, 1}
[[1.799e+01 1.038e+01 1.228e+02 1.001e+03 1.184e-01]
 [2.057e+01 1.777e+01 1.329e+02 1.326e+03 8.474e-02]
 [1.969e+01 2.125e+01 1.300e+02 1.203e+03 1.096e-01]]
[[ 1.09706398 -2.07333501  1.26993369  0.9843749   1.56846633]
 [ 1.82982061 -0.35363241  1.68595471  1.90870825 -0.82696245]
 [ 1.57988811  0.45618695  1.56650313  1.55888363  0.94221044]]


In [73]:
def data_split(X, y, ratio):
    ind = np.random.choice(len(y), int(ratio*len(y)), replace=False)
    ind = np.sort(ind)
    # print(ind)
    ind_trn = [i for i in range(len(y)) if i not in ind]
    X_trn, y_trn = X[ind_trn], y[ind_trn]
    X_tst, y_tst = X[ind], y[ind]
    return [X_trn, y_trn], [X_tst, y_tst]

# We do random sampling and save a seed for repeating results
np.random.seed(0)
trn, tst = data_split(X_n,y, ratio=0.2)
print(trn[0].shape)
print(tst[0].shape)

(456, 30)
(113, 30)


## Build a LR model
$y = f(x) = \sigma(\mathbf{w}^{T}\mathbf{x}+b), \mathbf{w} \in \mathrm{R}^{D}$ 

Let dataset be $\mathbf{X} \in \mathrm{R}^{N \times D}$, we have $\mathbf{x}_i \in \mathbf{X}, y_i = f(\mathbf{x}_i)$, which, in matrix form, is $\hat{\mathbf{y}} = \sigma(\mathbf{X}\mathbf{w}+b)$

Note that: $\mathbf{X}\mathbf{w}+b$ is equivalent to $[\mathbf{X}; \mathbf{1}] \mathbf{w}$, where $\mathbf{w} = [w_1,\dots,w_D,b] \in \mathrm{R}^{D+1}$.

$\nabla \mathbf{w} = \frac{1}{N} \sum_{i=1}^{N} (f(\mathbf{x}_i)-y_i)\mathbf{x}_i
=\frac{1}{N} \mathbf{X}^{T} (\hat{\mathbf{y}} - \mathbf{y})$

In [74]:
# Let’s random intialize w first
w0 = np.random.rand(D+1)
w0[-1] = 0
print(w0)
# Concatenate constant 1s to X_trn and X_tst
trn[0] = np.hstack((trn[0], np.ones((len(trn[0]),1))))
tst[0] = np.hstack((tst[0], np.ones((len(tst[0]),1))))
print(trn[0].shape)
print(tst[0].shape)

[0.09202759 0.05596583 0.08653249 0.23717329 0.83951297 0.52237053
 0.51307486 0.64983197 0.54459088 0.0324653  0.58015172 0.77108905
 0.37622657 0.49102474 0.98163968 0.24465141 0.3743232  0.04565964
 0.31006764 0.8315194  0.80702284 0.64002416 0.3681024  0.3127533
 0.80183615 0.07044719 0.68357296 0.38072924 0.63393096 0.92687909
 0.        ]
(456, 31)
(113, 31)


In [41]:
from scipy.special import expit
def sigmoid(z):
    return 1 / (1 + np.exp(-z))
def LR(X, w):
    return expit(np.matmul(X, w))

y_trn = LR(trn[0], w0)
print(y_trn)

delta_w = np.matmul(trn[0].T, (y_trn-trn[1])) / len(trn[0])
print(delta_w)

[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.

In [75]:
# Check the overflow issue
print(sigmoid(-1e3))
print(expit(-1e3))

0.0
0.0


<ipython-input-41-11d64e5169eb>:3: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-z))


In [99]:
def GD(dataset, w, lr=0.1, thres=1e-4, maxIter=400):
    w = np.copy(w)
    X,y = dataset
    i = 0
    while i<maxIter:
        y_pred = LR(X,w)
        delta_w = np.matmul(X.T, (y_pred-y)) / len(X)
        obj = np.max(np.absolute(delta_w))
        w -= lr*delta_w
        if obj < thres:
            break
        i += 1
    print(i)
    print(obj)
    return w
w_star = GD(trn, w0)
print(w0)
print(w_star)

400
0.007697934811666566
[0.09202759 0.05596583 0.08653249 0.23717329 0.83951297 0.52237053
 0.51307486 0.64983197 0.54459088 0.0324653  0.58015172 0.77108905
 0.37622657 0.49102474 0.98163968 0.24465141 0.3743232  0.04565964
 0.31006764 0.8315194  0.80702284 0.64002416 0.3681024  0.3127533
 0.80183615 0.07044719 0.68357296 0.38072924 0.63393096 0.92687909
 0.        ]
[-0.60681205 -0.81233552 -0.62808199 -0.51280545 -0.18209886 -0.07190057
 -0.48312732 -0.46592111 -0.26394842 -0.13603098 -0.62448847  0.1059247
 -0.64851371 -0.47380046  0.47432931  0.14777082  0.20195824 -0.37808033
  0.09256444  0.75678261 -0.19811697 -0.61561342 -0.59892091 -0.68297331
 -0.57285537 -0.60863784 -0.27491616 -0.74130261 -0.30815864  0.17792981
  0.61048118]


In [100]:
def com_acc(score, gnd):
    y_hat = np.zeros(score.shape)
    y_hat[score>0.5] = 1
    return (y_hat==gnd).sum() / len(gnd)
# prediction score on the training set
y_trn = LR(trn[0],w_star)
print("training acc={:.3f}".format(com_acc(y_trn, trn[1])))
# prediction score on the testing set
y_tst = LR(tst[0],w_star)
print("testing acc={:.3f}".format(com_acc(y_tst, tst[1])))

training acc=0.987
testing acc=0.956


In [88]:
# Let now see the sklearn implmentation
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(penalty='none', max_iter=400)
model.fit(trn[0],trn[1])

LogisticRegression(max_iter=400, penalty='none')

In [96]:
# Compute the accuracy with with Sklearn probability predictions
print("Sklern LR training acc={:.3f}".format(com_acc(model.predict_proba(trn[0])[:,1], trn[1])))
print("Sklern LR testing acc={:.3f}".format(com_acc(model.predict_proba(tst[0])[:,1], tst[1])))

Sklern LR training acc=1.000
Sklern LR testing acc=0.956


In [94]:
# Double the accuracy with with Sklearn-implemented functions
print("Sklern LR training acc={:.3f}".format(model.score(trn[0], trn[1])))
print("Sklern LR testing acc={:.3f}".format(model.score(tst[0], tst[1])))

Sklern LR training acc=1.000
Sklern LR testing acc=0.956
